In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error

In [2]:
df_aq = pd.read_csv("../input/beijing_1701_1803.csv")
df_station = pd.read_csv("../input/Beijing_AirQuality_Stations.csv")
target_cols = ['PM2.5', 'PM10', 'O3']

df_aq.utc_time = pd.to_datetime(df_aq.utc_time)

dfs = []
for stationId in df_aq.stationId.unique():
    df = df_aq[df_aq['stationId']==stationId]
    df = df.resample('1H', on='utc_time').sum()
    df = df.reset_index()
    df['stationId'] = stationId
    dfs.append(df)
df_ = pd.concat(dfs)

In [4]:
idx_to_station = {k:v for k,v in df_station.Station_ID.iteritems()}
near_stations = []
for k, row in df_station.iterrows():
    result =  np.sqrt((111*(df_station.longitude-116.417))**2 + (74*(df_station.latitude-39.929))**2).nsmallest(4)
    near_idx = np.sqrt((111*(df_station.longitude-row[1]))**2 + (74*(df_station.latitude-row[2]))**2).nsmallest(4).index[1:4].tolist()
    near_station = [idx_to_station[idx] for idx in near_idx]
    near_stations.append(near_station)
    print(k, row[0])
    print(result, "\n")
    
df_station_near = pd.concat([df_station, pd.DataFrame(near_stations, columns=['near_1', 'near_2', 'near_3'])], axis=1 )

0 dongsi_aq
0     0.000000
30    3.300267
1     3.370048
4     4.506715
dtype: float64 

1 tiantan_aq
0     0.000000
30    3.300267
1     3.370048
4     4.506715
dtype: float64 

2 guanyuan_aq
0     0.000000
30    3.300267
1     3.370048
4     4.506715
dtype: float64 

3 wanshouxigong_aq
0     0.000000
30    3.300267
1     3.370048
4     4.506715
dtype: float64 

4 aotizhongxin_aq
0     0.000000
30    3.300267
1     3.370048
4     4.506715
dtype: float64 

5 nongzhanguan_aq
0     0.000000
30    3.300267
1     3.370048
4     4.506715
dtype: float64 

6 wanliu_aq
0     0.000000
30    3.300267
1     3.370048
4     4.506715
dtype: float64 

7 beibuxinqu_aq
0     0.000000
30    3.300267
1     3.370048
4     4.506715
dtype: float64 

8 zhiwuyuan_aq
0     0.000000
30    3.300267
1     3.370048
4     4.506715
dtype: float64 

9 fengtaihuayuan_aq
0     0.000000
30    3.300267
1     3.370048
4     4.506715
dtype: float64 

10 yungang_aq
0     0.000000
30    3.300267
1     3.370048
4     4.506715

In [5]:
try:
    del df_impute
except:
    pass
df_impute = pd.merge(df_, df_station_near, 'left', left_on='stationId', right_on='Station_ID')
nears = ['near_1', 'near_2', 'near_3']
for near in nears:
    near_col = df_impute.iloc[0][near]
    df_right = df_impute[target_cols + ['stationId','utc_time']][df_impute.stationId==near_col]
    df_impute = pd.merge(df_impute, df_right, 
                         how='left', suffixes=('', '_'+ near),
                         left_on=['utc_time', near], 
                         right_on=['utc_time', 'stationId'])

In [6]:
def stKNN(df_impute, station):
    MSEs = []
    df_stKNNs_ = []
    df_impute_station = df_impute[df_impute.stationId==station]
    df_impute_station = df_impute_station.sort_values('utc_time')
    
    for target in target_cols:
        print(station, target)
        
        # Temperial 
        for i in [-2, -1, 1, 2]:
            df_impute_station[target + "_t{0:+}".format(i)] = df_impute_station[target].shift(i*-1)
            
        cols = [target + "_t{0:+}".format(i) for i in [-2, -1, 1, 2]]
        cols += [target+'_near_1', target+'_near_2', target+'_near_3', target, 'utc_time']
        df_stKNN_ = df_impute_station[cols]
        notnull_idxs = df_stKNN_[target].notnull().index # for evaluation

        # Iterate it until no null value
        while df_stKNN_[df_stKNN_[target].isnull()].shape[0]!=0:
            df_stKNN_[target+"_pred"] = df_stKNN_[cols].mean(axis=1)
            df_stKNN_.loc[df_stKNN_[target].isnull(), target] = df_stKNN_[target+'_pred']
            for i in [-2, -1, 1, 2]:
                df_stKNN_[target + "_t{0:+}".format(i)] = df_stKNN_[target].shift(i*-1)
            print(df_stKNN_[df_stKNN_[target].isnull()].shape[0], end=" ")

        df_stKNNs_.append(df_stKNN_[['utc_time', target]])
        mse = mean_absolute_error(df_stKNN_[target][notnull_idxs], df_stKNN_[target+'_pred'][notnull_idxs])
        MSEs.append([station + "_" + target, mse])

    df_stKNN = pd.concat(df_stKNNs_, axis=1)
    df_stKNN = df_stKNN.T.drop_duplicates().T
    df_stKNN['stationId'] = station
    return df_stKNN, MSEs
    
df_stKNN, MSEs = stKNN(df_impute, station="zhiwuyuan_aq" )

zhiwuyuan_aq PM2.5


/home/tony/venv/lib/python3.4/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/tony/venv/lib/python3.4/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/tony/venv/lib/python3.4/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

3967 3813 3724 3658 3600 3559 3526 3496 3466 3443 3423 3406 3392 3378 3364 3350 3336 3322 3308 3294 3280 3266 3252 3238 3225 3215 3205 3195 3185 3175 3165 3155 3145 3135 3125 3115 3105 3095 3085 3075 3065 3055 3049 3043 3037 3031 3025 3019 3013 3007 3001 2995 2989 2983 2977 2971 2965 2959 2953 2947 2941 2937 2935 2933 2931 2929 2927 2925 2923 2921 2919 2917 2915 2913 2911 2909 2907 2905 2903 2901 2899 2897 2895 2893 2891 2889 2887 2885 2883 2881 2879 2877 2875 2873 2871 2869 2867 2865 2863 2861 2859 2857 2855 2853 2851 2849 2847 2845 2843 2841 2839 2837 2835 2833 2831 2829 2827 2825 2823 2821 2819 2817 2815 2813 2811 2809 2807 2805 2803 2801 2799 2797 2795 2793 2791 2789 2787 2785 2783 2781 2779 2777 2775 2773 2771 2769 2767 2765 2763 2761 2759 2757 2755 2753 2751 2749 2747 2745 2743 2741 2739 2737 2735 2733 2731 2729 2727 2725 2723 2721 2719 2717 2715 2713 2711 2709 2707 2705 2703 2701 2699 2697 2695 2693 2691 2689 2687 2685 2683 2681 2679 2677 2675 2673 2671 2669 2667 2665 2663 2661 

KeyboardInterrupt: 

In [7]:
station="zhiwuyuan_aq"
MSEs = []
df_stKNNs_ = []
df_impute_station = df_impute[df_impute.stationId==station]
df_impute_station = df_impute_station.sort_values('utc_time')
target="PM2.5"

print(station, target)

# Temperial 
for i in [-2, -1, 1, 2]:
    df_impute_station[target + "_t{0:+}".format(i)] = df_impute_station[target].shift(i*-1)

cols = [target + "_t{0:+}".format(i) for i in [-2, -1, 1, 2]]
cols += [target+'_near_1', target+'_near_2', target+'_near_3', target, 'utc_time']
df_stKNN_ = df_impute_station[cols]
notnull_idxs = df_stKNN_[target].notnull().index # for evaluation

# Iterate it until no null value
# while df_stKNN_[df_stKNN_[target].isnull()].shape[0]!=0:
#     df_stKNN_[target+"_pred"] = df_stKNN_[cols].mean(axis=1)
#     df_stKNN_.loc[df_stKNN_[target].isnull(), target] = df_stKNN_[target+'_pred']
#     for i in [-2, -1, 1, 2]:
#         df_stKNN_[target + "_t{0:+}".format(i)] = df_stKNN_[target].shift(i*-1)
#     print(df_stKNN_[df_stKNN_[target].isnull()].shape[0], end=" ")

# df_stKNNs_.append(df_stKNN_[['utc_time', target]])
# mse = mean_absolute_error(df_stKNN_[target][notnull_idxs], df_stKNN_[target+'_pred'][notnull_idxs])
# MSEs.append([station + "_" + target, mse])


zhiwuyuan_aq PM2.5


In [ ]:
# a, b = stKNN(df_impute, station="zhiwuyuan_aq")